# P4 Tips
---

## 加载数据
*  使用字典形式创建图片处理方式：  
data_transforms = {  
   'train' : .... ,  
   'valid' : .... ,   
   'test' : ....      }
   
* transforms.Compose([
    * 随机裁剪
        transforms.RandomResizedCrop(), 
        
    * 随机翻转
        transforms.RandomHorizontalFlip(),
    * 随机旋转
        transforms.RandomRotation(), 
    * 将 PIL.Image/numpy.ndarray 数据进转化为torch.FloadTensor，并归一化到[0, 1.0]
        transforms.ToTensor(), 
    * 将均值和标准差标准化到网络期望的结果  
    transforms.Normalize(expect_mean, expect_std)])

*  ‘valid’和‘test’数据集最好将图片调整至256*256后再进行裁剪：  
   transforms.Resize(256)

* images_datasets 和 dataloaders  均可使用 data_transforms 的键进行简化书写：
   image_datasets = {x: ...... for x in list(data_transforms.keys( ) ) }  
   dataloaders = {x: ...... for x in list(data_transforms.keys( ) ) } 

* 值得注意的是，dataloaders 中的 batch_size 设置不宜过大，以32为宜，同时要设置 shuffle=True 将数据随机化
---

## 标签映射
* cat_to_name 的长度是102，即共计102类花卉，也是输出的维度。

## 构建和训练分类器
---
### 加载预训练的网络
* 推荐使用 densenet121，分类器（全连接层）输入层单元数较少（1024），同时设置1层隐藏层，隐藏层单元数介于1024和102即可，训练较快

### 修改全连接层
* 使用 classifier = nn.Sequential(OrderedDict([    ])) 进行修改
*  输出层单元数为花卉种类（102），即 len(cat_to_name)
*  对于此项目，不建议使用 dropout，小数据集往往是欠拟合，不用 dropout 能减少训练时间
* 根据损失函数，全连接层的最后一层需做相应修改，推荐使用交叉熵损失函数，全连接层可有效简化
---
### 训练网络
* 在训练网络的函数中，需要确定损失函数（criterion）和优化器（optimizer），优化器常规可使用 Adam
* 通过 device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 判断是否能使用 GPU 模式
* 训练函数实现的功能：
   * 输入：超参数（训练周期，学习速率等）
   * 输出：训练损失及正确率，验证损失及正确率
* 值得注意的细节：
   * 模型输入输出的数据维度
   * 每 1 次损失函数输出的是 batch_size 张图片的平均损失，分阶段输出预测正确率时需要注意基数的大小
   * 进入训练模式、验证模式需要特定语句
* 训练函数的流程大致如下：
   * 确定必要的函数输入（超参数）
   * 加载已修改好分类器的模型
   * 将模型送入 GPU
   * 指定损失函数和优化器，其中优化器需要求确定优化对象（model.classifier.parameters()）和学习速率（lr）
   * 对于每 1 个训练周期：
      * 设置必要的计数变量（损失和正确率等）
      * 使用 model.train() 进入训练模式
      * 每次从 dataloaders 中的训练集获取 1 批数据，数据包含 2 个内容，分别是图片的像素和标签（对应的种类）：images，labels。images 的维度：[32, 3, 224, 224]，32 是 batch_size, 意味着每次进行计算的图片数量为 32； 3 是图片通道数，224 * 224 即进行中心裁剪后图片的像素个数；labels 的维度是 32，内容是每张图对应的标签。
      * 将 images 和 labels 送入 GPU
      * 将梯度初始化为零
      * 将 images 作为输出送入模型，获取输出 output （模型的输入维度为32 * 3 * 224 * 224，由之前的 data_fransforms 已完成预处理；output 的维度是[32,102]，意味着 32 张图中每 1 张对应 102 种分类的“激励值”）
      * 获取最大"激励值"对应的标签（此为模型预测的标签，preds）
      * 将 output 和 labels 送入损失函数计算 loss
      * 反向传播 loss
      * 进行优化
      * 累加训练损失（需要注意：通过 loss.item() 从张量中获得 python number，否则内存会被一直消耗而得不到释放https://zhuanlan.zhihu.com/p/36307662）
      * 通过图片原标签（labels）与预测标签（preds）相比较确定预测正确率
      * 输出训练损失及正确率（为了更好地跟踪训练过程，可在 1 个训练周期中多次输出损失和正确率）

    * 进入验证模式 model.eval()
    * 关闭梯度计算 with torch.no_grad(): 
      * 验证损失和正确率的计算与上相似，不再赘述（为了节省运算时间，可在每 1 次训练周期结束后进行验证输出）
          
  * 返回训练好的模型
---
### 测试网络
  * 获取 dataloaders 中测试集的数据，按验证方法进行测试

### 保存检查点
  * 以字典的方式保存以下内容（不限于）：
    * 训练好的模型参数
    * 已更改的分类器结构
    * 从类别到索引的映射
  * 保存后在当前路径下会生成一个.pth 文件，可下载到本地
---
### 加载检查点
  * 将保存的模型信息一一加载到新的模型中，模型种类需要匹配
  * 加载成功后的模型应与之前训练好的模型一致，可用此对新数据进行预测
---

## 类别推理
---
### 图像处理
* process_image 函数输入的是图片的路径，可以使用 data_transforms 中测试集的图像处理方式对输入图片进行处理，返回的是一个 Tensor，目的是将此 Tensor 作为输入送入已训练好的模型进行预测
* imshow 函数输入的是一个图片的 Tensor，返回的是一张图片，作用是图片的可视化
---
### 类别预测
* process_image 的输出维度是 [3, 224,224]，与模型输入[32, 3, 224, 224] 不一致，需要使用 Tensor.unsqueeze(0) 匹配成 [1, 3, 224, 224]
* 模型输出后还不是概率值，需要归一化为概率，根据损失函数的不同进行相应变换
* 获取的概率值(probs)和相应的标签（labels）从 Tensor 转换为 numpy ，需要进行如下操作：
  * probs.cpu().detach().numpy().tolist()[0] 
  * labels.cpu().detach().numpy().tolist()[0] 
---
### 检查运行状况
* 选择恰当的方式将图片预测出的标签值映射到花卉名称，并与相应的概率值作为二维数据进行作图（推荐使用 seaborn.barplot() ）
* 图片路径中（例如 'flowers/test/100/image_07902.jpg'）100为标签，可使用 cat_to_name[100] 获取花卉的真实名称
---

## 参考链接：
*  https://github.com/cjimti/aipnd-project
* https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#load-data
* https://medium.com/@josh_2774/deep-learning-with-pytorch-9574e74d17ad
* https://github.com/miraclewkf/ImageClassification-PyTorch/blob/master/level1/train.py
* http://milletpu.com/2018/04/07/pytorch-view/
* https://zhuanlan.zhihu.com/p/34395749
* https://discuss.pytorch.org/t/any-different-between-model-input-and-model-forward-input/3690
* https://cosx.org/2017/10/transfer-learning/
* https://hit-scir.gitbooks.io/neural-networks-and-deep-learning-zh_cn/content/chap3/c3s1.html
